#LDA (비지도 학습)

In [1]:
# 토픽 모델링, 문서의 단어 분포 패턴에서 토픽을 추출하는 알고리즘
# LDA는 문서가 토픽의 혼합으로 구성되어 있으면, 토픽은 각각의 단어 분포를 가지고 있다고 가정
# 각 단어에 어떤 토픽에서 생성되었을지를 추첮앟여 문서의 주제를 파악

In [2]:
# 1. 토픽의 갯수를 지정 (전체 글에서 비슷한 글끼리 묶어줌)
# 2. 토픽의 혼합 비율을 무작위로 설정
# 3. 모든 단어를 특정 토픽에 할당
# 4. 아래 작업 반복
#    1) 현재 문서의 모든 단어에 대해 해당 단어가 속한 토픽 할당
#    2) 현재 문서에서 각 토픽이 할당된 비율 업데이트
#    3) 모든 문서에서 해당 토픽이 할당된 비율 업데이트
# 각 문서에서 토픽이 할당된 비율을 이용하여 해당 문서의 주제를 추정

In [3]:
# 댓글 분석 or SNS 상의 데이터 분석

In [4]:
!pip install konlpy

In [5]:
# LDA 시각화 패키지
!pip install -U pyLDAvis

### 데이터 로드

In [32]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/메타버스_아카데미_2기/딥러닝/7월/data/app_stadew_valley_review.csv',index_col=0)
df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,comment,score
0,스타듀밸리 재미있게 하고있습니다!! 이번에 업데이트하면서 그런건지 모르겠지만 농장 ...,5
1,게임을 할 때 화면을 확대하면 화면이 깨지는 것처럼 작은 실선과 같은 모양으로 지직...,4
2,게임을 잘 플레이하고있으나 업데이트 이후로 생긴 문제점이 있어 다음 업데이트에 참고...,5
3,안녕하세요. 업데이트 이후 버그가 생겼습니다. 1. 주민과 대화할 때 대화창과 글씨...,5
4,저만 그런지 모르겠습니다만 업데이트 전 까지만 해도 잘 저장되던 세이브파일이 업데이...,3
...,...,...
2755,언제 업댓되나요 ㅜㅜ,5
2756,아주 재밌습니다 인생겜,5
2757,2번구매됬는데 1번 환불이 왜 안되죠,1
2758,말해 뭐합니까 보증된 게임,5


In [33]:
# df = df.iloc[3:]
# df = df.reset_index()
# df = df.drop(columns=['index','Unnamed: 0'])
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,comment,score
0,스타듀밸리 재미있게 하고있습니다!! 이번에 업데이트하면서 그런건지 모르겠지만 농장 ...,5
1,게임을 할 때 화면을 확대하면 화면이 깨지는 것처럼 작은 실선과 같은 모양으로 지직...,4
2,게임을 잘 플레이하고있으나 업데이트 이후로 생긴 문제점이 있어 다음 업데이트에 참고...,5
3,안녕하세요. 업데이트 이후 버그가 생겼습니다. 1. 주민과 대화할 때 대화창과 글씨...,5
4,저만 그런지 모르겠습니다만 업데이트 전 까지만 해도 잘 저장되던 세이브파일이 업데이...,3


### 형태소 추출

In [34]:
import konlpy
import re
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
def tokenize_text(text):
  text = re.sub(r'[^ㄱ - | 가-힣\s]','',text) # \s : 공백

  okt = konlpy.tag.Okt()
  okt_morphs = okt.pos(text)

  words = []

  for word,pos in okt_morphs:

    # 명사 동사 형용사만
    if pos in ['Adjective','Verb','Noun']:

      # LDA는 문서형태로 넣어야함
      words.append(word)

  word_str = ' '.join(words)
  return word_str

tokenize_text("스타듀밸리 너무 재밌는 힐링 게임입니다!")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'스타 듀 밸리 재밌는 힐링 게임 입니다'

In [38]:
token_list = []
for text in tqdm(df['comment']):
  token_list.append(tokenize_text(text))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 2760/2760 [00:15<00:00, 180.59it/s]


In [39]:
token_list[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


'스타 듀 밸리 재미있게 하고있습니다 이번 업데이트 하면서 건 모르겠지만 농장 컴퓨터 미스터 치 방 볼수있는 진행 등등 창 확대 되서 읽는 부분 있습니다 그 외 게임 키 처음 아이템 창 커져있을 때 있습니다 게임 설정 들어갔다가 나오면 해결 됩니다만 매번 그러니 불편함이 있네요 개선 부탁드립니다'

In [40]:
corpus_list = []
for idx in range(len(token_list)):
  corpus = token_list[idx]
  # 짧은 문장들만 저장
  # 짧은 문장안의 몇개안되는 단어들로만 연산하기 때문에 오분류할 수 있음
  if len(set(corpus.split()))<3:
    corpus_list.append(corpus)
corpus_list

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['화면 작아요',
 '에비게일 좋아여',
 '업데이트 업데이트',
 '귀엽고 재밌어요',
 '재미있게하고있어요',
 '셰인 귀여워',
 '도트 귀여워',
 '재미있습니다 인생게임',
 '감 불편',
 '재밌어요 추천',
 '노동 밸리',
 '재밌어요',
 '재미있고 좋아요',
 '멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀 멀티 좀',
 '한국어 없나요'

In [41]:
# 만약 짧은 문장이 corpus_list에 있다면 제거
for corpus in corpus_list:
  token_list.remove(corpus)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
# LDA는 count 기반의 vectorizer만 사용!

# max_df : 0.1, 전체 문서에서 10%이상 나오는 단어 무시
# min_df : 2, 2개 미만 문서에서 나오면 무시
# ngram_range = (1,2), 1개 or 2개로 이루어진 단어
# https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer
count_vectorizer = CountVectorizer(max_df=0.1,min_df=2,max_features=2000,ngram_range=(1,4))

feat_vect = count_vectorizer.fit_transform(token_list)
feat_vect.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2717, 2000)

In [44]:
feat_vect.toarray()[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([0, 0, 0, ..., 0, 0, 0])

In [61]:
len(feat_vect.toarray()[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2000

In [45]:
count_vectorizer.vocabulary_

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'스타': 891,
 '밸리': 688,
 '재미있게': 1379,
 '하고있습니다': 1819,
 '이번': 1242,
 '하면서': 1861,
 '모르겠지만': 596,
 '농장': 324,
 '컴퓨터': 1722,
 '진행': 1634,
 '등등': 467,
 '확대': 1974,
 '되서': 432,
 '부분': 749,
 '있습니다': 1317,
 '처음': 1657,
 '아이템': 994,
 '설정': 850,
 '나오면': 282,
 '해결': 1901,
 '매번': 530,
 '그러니': 210,
 '있네요': 1302,
 '개선': 62,
 '부탁드립니다': 753,
 '스타 밸리': 892,
 '이번 업데이트': 1243,
 '업데이트 하면서': 1107,
 '있습니다 게임': 1318,
 '게임 처음': 121,
 '개선 부탁드립니다': 63,
 '이번 업데이트 하면서': 1245,
 '화면': 1969,
 '하면': 1858,
 '작은': 1364,
 '실선': 932,
 '같은': 52,
 '거리': 71,
 '보입니다': 738,
 '번만': 715,
 '확인': 1976,
 '축소': 1690,
 '상태': 831,
 '손가락': 863,
 '않고': 1027,
 '움직': 1208,
 '기능': 226,
 '없어져서': 1142,
 '아쉽습니다': 990,
 '새로운': 835,
 '캐릭터': 1711,
 '낚시': 292,
 '했더니': 1943,
 '하는': 1829,
 '보이는': 736,
 '물고기': 639,
 '왔다': 1198,
 '갔다': 34,
 '되어': 433,
 '제대로': 1496,
 '없었습니다': 1144,
 '이건': 1229,
 '글씨': 217,
 '자체': 1358,
 '커져서': 1716,
 '생각': 838,
 '조금': 1512,
 '크기': 1729,
 '있을까요': 1339,
 '빠른': 783,
 '수정': 870,
 '화면 확대': 1971,
 '낚시 하는': 298,
 '빠른 수정'

In [46]:
lda = LatentDirichletAllocation(n_components=5)
lda.fit(feat_vect)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=5)

In [47]:
feature_names = count_vectorizer.get_feature_names_out()
feature_names

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['가게', '가격', '가구', ..., '힘들게', '힘들고', '힘들어요'], dtype=object)

In [48]:
# 토픽을 5개로 설정함
# 높은 수가 가중치가 높은 단어를 뜻함
lda.components_

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[ 0.33430533, 23.87758292,  5.86933206, ...,  4.07373406,
         0.21054237,  8.8176408 ],
       [ 0.20001206,  4.13907185,  0.20092711, ...,  0.20000864,
         2.37237402,  3.55486185],
       [ 4.07958619,  1.57806684,  0.20000428, ...,  0.20068032,
         0.20124812,  0.20104636],
       [ 4.18464251,  0.20198861,  0.52973258, ...,  3.12053455,
         5.0158277 ,  0.22467708],
       [ 0.20145391,  0.20328979,  0.20000397, ...,  0.40504243,
         0.20000779,  0.20177391]])

In [49]:
len(lda.components_[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2000

### LDA 분류한 토픽별 가중치가 높은 단어들 추출

In [50]:
def display_topics(model,feature_names,num_top_words=10):
  for topic_index,topic in enumerate(model.components_):
    print("Topic : ",topic_index)
    topic_word_indexes = topic.argsort()[::-1]
    top_indexes = topic_word_indexes[:num_top_words]
    print("top_indexes : ",top_indexes)
    feature_concat = ''.join([feature_names[i] for i in top_indexes])
    print("feature_concat : ",feature_concat)

display_topics(lda,feature_names,20)

Topic :  0
top_indexes :  [ 695  353 1177  145 1969 1448 1246 1711  870  994 1899  847  851  160
  185 1704  907  418 1916  921]
feature_concat :  버그다시오류계속화면저장이벤트캐릭터수정아이템합니다선택설치고쳐주세요구매카지노시간되는해서시작
Topic :  1
top_indexes :  [1552 1112 1924 1497 1532 1474 1618 1077  425   49  574  602  588 1697
 1528  609  709 1678 1554 1422]
feature_concat :  좋은데업뎃해주세요제발좋겠어요정말진짜언제되면같아요멀티플레이모바일 멀티모드친구좋겠네요모바일 업데이트버전추가좋은데 멀티재밌는데
Topic :  2
top_indexes :  [1777  226  907 1807 1405 1678 1762  538 1711 1303 1916 1474 1969  855
  353 1829 1317 1729 1618 1899]
feature_concat :  플레이기능시간하고재밌게추가파일멀티 기능캐릭터있는해서정말화면세이브다시하는있습니다크기진짜합니다
Topic :  3
top_indexes :  [ 292 1618 1980 1924 1433 1858 1376 1939 1515  150 1165 1512  907 1657
 1139 1472  488 1840 1310 1497]
feature_concat :  낚시진짜환불해주세요재밌어요하면재미했는데조작계정연동조금시간처음없어서정도리텍하다있는데제발
Topic :  4
top_indexes :  [ 891  688  892 1287 1991 1777 1829 1474  907 1220  802 1618  588  882
  105  324  709 1992 1428 1899]
feature_concat :  스타밸리스타 밸리입니다힐링플레이하는정말시간유저사람진짜모드스듀게임 입니다농장버전힐링 게임재

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


###LDA 시각화

In [51]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda,feat_vect,count_vectorizer)
pyLDAvis.display(vis)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 각 문서별 높은 확률의 토픽

In [52]:
sent_topic = lda.transform(feat_vect)

# 0번 문장에 대한 토픽 확률
# 총합이 1 (softmax)

sent_topic[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([0.46308446, 0.00582139, 0.17575635, 0.00582497, 0.34951283])

In [53]:
len(sent_topic)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2717

In [54]:
sent_topic.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(2717, 5)

In [55]:
sent_topic_per_list = []
for n in range(sent_topic.shape[0]):
  sent_topic_list = sent_topic[n].argmax()
  topic_per = sent_topic[n].max()
                            # 수, 토픽분류     , 가장 높은 토픽일 확률
  sent_topic_per_list.append([n,sent_topic_list,topic_per])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
topic_per_df = pd.DataFrame(sent_topic_per_list,columns=['no','토픽번호','확률'])
topic_per_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,no,토픽번호,확률
0,0,0,0.463084
1,1,0,0.987717
2,2,2,0.977528
3,3,0,0.463406
4,4,0,0.562169
...,...,...,...
2712,2712,1,0.732620
2713,2713,4,0.799419
2714,2714,0,0.796972
2715,2715,0,0.200000


### 데이터프레임 결합

In [57]:
doc_topic_df = topic_per_df.join(df)
doc_topic_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,no,토픽번호,확률,comment,score
0,0,0,0.463084,스타듀밸리 재미있게 하고있습니다!! 이번에 업데이트하면서 그런건지 모르겠지만 농장 ...,5
1,1,0,0.987717,게임을 할 때 화면을 확대하면 화면이 깨지는 것처럼 작은 실선과 같은 모양으로 지직...,4
2,2,2,0.977528,게임을 잘 플레이하고있으나 업데이트 이후로 생긴 문제점이 있어 다음 업데이트에 참고...,5
3,3,0,0.463406,안녕하세요. 업데이트 이후 버그가 생겼습니다. 1. 주민과 대화할 때 대화창과 글씨...,5
4,4,0,0.562169,저만 그런지 모르겠습니다만 업데이트 전 까지만 해도 잘 저장되던 세이브파일이 업데이...,3
...,...,...,...,...,...
2712,2712,1,0.732620,멀티플레이가되면 좋겠음니다,5
2713,2713,4,0.799419,달걀축제때 달걀 안먹어지는 버그좀 고쳐주세요,4
2714,2714,0,0.796972,템증발 버그 있습니다 확인해주세요,3
2715,2715,0,0.200000,아기 잘 낳았어요,5


In [58]:
doc_topic_df['토픽번호'].value_counts()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


토픽번호
1    669
0    603
4    512
3    493
2    440
Name: count, dtype: int64

### 토픽별 확률이 높은 데이터를 찾아 어떤 토픽인지 라벨링

In [60]:
from pandas.core.algorithms import doc
# topic = 0,1,2,3,4
for topic in range(len(doc_topic_df['토픽번호'].unique())):
  print("Top No : ",topic)
  top_per_topics = doc_topic_df[doc_topic_df['토픽번호']==topic].sort_values(by='확률',ascending=False)
  print(top_per_topics['comment'].iloc[0])
  print(top_per_topics['comment'].iloc[1])
  print(top_per_topics['comment'].iloc[2])

Top No :  0
정말 힐링되고 좋지만 오류가 잦아서 불편합니다 이벤트나 캐릭터들과의 대화는 참을수있지만 가끔이 아이템들이 사라지는 오류는 꼭좀 고쳐주셨으면 합니다. 힘들게 얻은 아이템이 갑자기 사라지니 너무 놀라고 충격이라 아예 하루를 다시 시작하기도 합니다. 가장 최근에는 마동석에서 나온 불의 석영을 번들에 쓰려고 다른 보물들만 기증하고 보물들의 량을 보느라 시간이 살짝 흐르고 ×를 누르고 나왔는데 기증도 안한 불의 석영이 없었고 기증하는 곳에도 불의 석영이 없고 아예 증발이 됐습니다. 그자리에서 저장하고 나갔다 다시 들어오니 기증할때의 화면이 뜬 상태로 화면만 움직이고 ×버튼도 없어서 캐릭터도 조종을 못해 결국 하루를 다시시작하게 됐습니다. 하루를 다시시작하는건 오류가 잦음으로 인해 이젠 익숙해졌지만 익숙해졌다고 마음이 편한건 아닙니다. Pc기능들이 모바일로 오기를 기다리고 있지만 그전에 이런 잦은 오류들을 부디 고쳐주세요. 너무 잦아서 지칩니다. 이동,모션방향,아이템증발,대화 오류 등등...
처음엔 지루했지만 하면 할수록 자유도도 꽤 높고 재밌었음. 돈이 아깝지 않은 최고의 게임
게임을 할 때 화면을 확대하면 화면이 깨지는 것처럼 작은 실선과 같은 모양으로 지직거리는 것처럼 보입니다. 저만 그런진 모르겠지만 한 번만 확인 부탁드립니다. 또, 업데이트 전에는 화면을 축소한 상태로 손가락을 떼지 않고 옮기면 화면이 움직여서 시야를 더 넓게 볼 수 있었는데요 업데이트 후에 이 기능이 없어져서 아쉽습니다. 또, 이번에 새로운 캐릭터를 파서 낚시를 했더니 화면을 확대하면 낚시하는 과정 중 보이는 가운데 막대에 물고기 모양이 왔다갔다 하는 바가 같이 확대되어 화면을 넘어서서 낚시를 제대로 할 수 없었습니다. 그리고 이건 정말정말 죄송한데 캐릭터 말풍선의 글씨가 작아서 잘 안 보인다고 올렸었는데요, 말풍선 자체가 커져서 아이템창 선을 넘어섰고 생각보다 글씨가 조금 많이 커져서 그런데 업데이트 전과 같은 크기로 조금만 줄여주실 수 있을까요?ㅠㅠ빠른 수정 부탁드립니

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
